[Chapter 6] Feature Importance Analysis

In [4]:
import numpy as np
import pandas as pd

from sklearn.datasets import make_classification
import statsmodels.discrete.discrete_model as sm

import seaborn as sns

motivation - a numerical example

In [6]:
def getTestData(n_features=100, n_informative=25, n_redundant=25, n_samples=10000, random_state=0, sigmaStd=.0):
    """
    generate a random dataset of informed, redundant, and noise explanatory variables for a classification problem
    """
    np.random.seed(random_state)

    X, y = make_classification(n_samples=n_samples, n_features=n_features-n_redundant,
        n_informative=n_informative, n_redundant=0, shuffle=False, random_state=random_state)
    
    # I_: informative features (used to generate labels)
    cols = ['I_'+str(i) for i in range(n_informative)]
    
    # N_: noise features (that are not used to generate labels)
    cols += ['N_'+str(i) for i in range(n_features-n_informative-n_redundant)]

    # R_: redundant features
    # formed by adding Gaussian noise to a randomly chosen informative feature
    X, y = pd.DataFrame(X, columns=cols), pd.Series(y)
    # randomly choosen informative feature
    i = np.random.choice(range(n_informative), size=n_redundant)
    # the lower the sigmaStd, the greater the substitution effect (cannot discriminate among redundant variables)
    for k, j in enumerate(i):
        X['R_'+str(k)] = X['I_'+str(j)] + np.random.normal(size=X.shape[0])*sigmaStd 
    
    return X, y

X, y = getTestData(40, 5, 30, 10000, sigmaStd=0.1)
ols = sm.Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.380271
         Iterations 7


feature importance